# RAG + LLM -> 퀴즈, 해설 생성
- LLM : GPT-oss 20b 사용


In [2]:
import torch
print("cuda available:", torch.cuda.is_available())
print("torch version:", torch.__version__)
print("cuda version reported by torch:", torch.version.cuda)
if torch.cuda.is_available():
    print("gpu:", torch.cuda.get_device_name(0))


cuda available: True
torch version: 2.6.0+cu124
cuda version reported by torch: 12.4
gpu: NVIDIA GeForce RTX 4060 Laptop GPU


## vector 임베딩 후 검색하는 코드 (기본)

### 상황 하나 당 여러 문제


In [ ]:
# ===== 1. 라이브러리 임포트 =====
import os
import warnings
from pprint import pprint

import torch
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.llms import Ollama
from langchain_core.prompts import PromptTemplate
from langchain.schema import Document

# 모든 경고 무시
warnings.filterwarnings("ignore")

# ✅ 디바이스 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# ===== 캐시 저장 경로 =====
VECTORSTORE_PATH = "./vectorstore_cache"

# ===== 2. PDF 로드 (첫 실행 시에만) =====
pdf_path = r"도로교통법(법률)(제20677호).pdf"

# ===== 4. 임베딩 모델 =====
embedding = HuggingFaceEmbeddings(
    model_name="intfloat/multilingual-e5-base",
    model_kwargs={"device": "cuda"},  #if torch.cuda.is_available() else "cpu"},
    encode_kwargs={"normalize_embeddings": True},
)


# ===== 5. 벡터스토어 로드 또는 생성 =====
if os.path.exists(VECTORSTORE_PATH):
    print("✅ 기존 벡터스토어 로드 중...")
    vectorstore = FAISS.load_local(
        VECTORSTORE_PATH,
        embedding,
        allow_dangerous_deserialization=True
    )
else:
    print("⚡ 신규 벡터스토어 생성 중...")
    loader = PyPDFLoader(pdf_path)
    pages = loader.load()

    # 문서 분할
    text_splitter = RecursiveCharacterTextSplitter(
        separators=["제", "조", "\n\n", "\n"],
        chunk_size=1200,
        chunk_overlap=200
    )
    splits = text_splitter.split_documents(pages)

    # E5 권장 프리픽스 적용
    splits_e5 = [
        Document(page_content="passage: " + d.page_content, metadata=d.metadata)
        for d in splits
    ]

    # FAISS 인덱스 생성
    vectorstore = FAISS.from_documents(splits_e5, embedding)

    # 로컬 저장
    vectorstore.save_local(VECTORSTORE_PATH)
    print(f"💾 벡터스토어 저장 완료: {VECTORSTORE_PATH}")

# ===== 6. 리트리버 설정 =====
retriever = vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 10, "fetch_k": 20}
)

# ===== 7. 질문(Query) =====
query = """

에고 차량은 3차선 도로의 중간 차선을 주행합니다. 
에고 차량이 차선을 바꿔 버스 전용차로로 주행합니다.
보행자가 갑자기 무단횡단하여 버스 전용차로 도로로 들어서고, 자아 차량과 보행자 간의 충돌이 발생합니다. 
보행자가 바닥에 쓰러지고 에고 차량은 주행을 멈춥니다.


"""

# ===== 8. 관련 문서 검색 =====
search_result = retriever.get_relevant_documents(f"query: {query.strip()}")

# ===== 9. 검색 결과 미리보기 =====
def pretty_print_documents(doc_list):
    for i, doc in enumerate(doc_list):
        print(f"\n📄 문서 {i+1}")
        print(f"▶ 페이지 번호 : {doc.metadata.get('page_label', doc.metadata.get('page'))}")
        print("▶ 전체 내용:")
        print(doc.page_content)
        print("-" * 80)

pretty_print_documents(search_result)

# ===== 10. LLM 모델 =====
llm = Ollama(model="gpt-oss")

# ===== 11. 프롬프트 템플릿 =====
template = """
당신은 도로교통법 전문가입니다.
다음은 도로교통법 문서에서 검색된 일부 조항과 문장입니다:

{context}

아래 도로주행상황설명 텍스트에 대해 도로교통법 조문을 인용해 객관식 문제를 생성하세요.

요구사항:
1. 도로주행상황설명 텍스트는 영상을 시간순으로 보며 운전자 시점에서 벌어지는 상황을 나열한 것입니다. 운전자 시점에서 상황을 이해하세요.
2. 도로 주행 시 운전자가 마주하는 사고 위험 순간이 있습니다. 사고 위험 순간을 리스트로 정리해주세요.
3. 사고 위험 순간 리스트의 케이스를 중 하나를 골라, 만약 문제를 푸는 사람이 운전자라면 사고 위험 순간 직전에 어떻게 행동하는 것이 올바른 행동인지 맞추는 객관식 문제를 만들어주세요.
4. 객관식 문제의 선택지는 정답(운전자의 올바른 행동) 1개, 오답(운전자의 잘못된 행동) 3개로 만들어주세요.
5. 운전자의 올바른 행동은 도로교통법 조문에 근거한 행동이어야 합니다. 운전자의 잘못된 행동은 위법,위협,사고유발 행동이어야 합니다.
6. 문제와 답을 만든 후, 4개의 선택지에 대해 해설을 해주세요. 
7. 정답 선지의 경우, 왜 정답인지 도로교통법 조문을 인용해 "도로교통법 제XX조 제X항 '문장 전체'"를 반드시 그대로 인용 후 정답인 이유를 설명하세요.
8. 오답 선지의 경우, 왜 오답인지 도로교통법 조문을 인용해 "도로교통법 제XX조 제X항 '문장 전체'"를 반드시 그대로 인용 후 오답인 이유를 설명하세요.
9. 위 작업을 사고 위험 순간 리스트에 있는 모든 케이스에 대해 적용해주세요. 케이스 1개 당 객관식 문제 1개를 만들어주세요. 만약 케이스가 8개라면 객관식 문제 8개가 나와야 합니다.
10. 답변은 한국어로 해주세요.
11. 위 요구사항을 전부 지켜주세요.



질문:
{question}

"""
prompt = PromptTemplate.from_template(template)

# ===== 12. 최종 프롬프트 생성 =====
context = "\n\n".join(doc.page_content for doc in search_result)
final_prompt = prompt.format(context=context, question=query)

# ===== 13. Ollama 호출 =====
response = llm.invoke(final_prompt)

print(f"\n\n======= Ollama 모델 : {llm.model} 응답 =======")
print(response)

Using device: cuda
✅ 기존 벡터스토어 로드 중...

📄 문서 1
▶ 페이지 번호 : 7
▶ 전체 내용:
passage: 제18조의2제1항 각 호에 따른 도로 또는 차로를 말한다.
8. “자전거도로”란 안전표지, 위험방지용 울타리나 그와 비슷한 인공구조물로 경계를 표시하여 자전거 및 개인형 이
동장치가 통행할 수 있도록 설치된 「자전거 이용 활성화에 관한 법률」 제3조 각 호의 도로를 말한다.
9. “자전거횡단도”란 자전거 및 개인형 이동장치가 일반도로를 횡단할 수 있도록 안전표지로 표시한 도로의 부분을
말한다.
10. “보도”(步道)란 연석선, 안전표지나 그와 비슷한 인공구조물로 경계를 표시하여 보행자(유모차, 보행보조용 의자
차, 노약자용 보행기 등 행정안전부령으로 정하는 기구ㆍ장치를 이용하여 통행하는 사람 및 제21호의3에 따른 실
외이동로봇을 포함한다. 이하 같다)가 통행할 수 있도록 한 도로의 부분을 말한다.
11. “길가장자리구역”이란 보도와 차도가 구분되지 아니한 도로에서 보행자의 안전을 확보하기 위하여 안전표지 등
으로 경계를 표시한 도로의 가장자리 부분을 말한다.
12. “횡단보도”란 보행자가 도로를 횡단할 수 있도록 안전표지로 표시한 도로의 부분을 말한다.
13. “교차로”란 ‘십’자로, ‘T’자로나 그 밖에 둘 이상의 도로(보도와 차도가 구분되어 있는 도로에서는 차도를 말한다
)가 교차하는 부분을 말한다.
--------------------------------------------------------------------------------

📄 문서 2
▶ 페이지 번호 : 19
▶ 전체 내용:
passage: 법제처                                                            19                                                       국가법령정보센터
도로교통법
제23조(끼어들기의 금지) 모든 차의 운전자는 제22조제2항 각 호의 

### 상황 하나 당 문제 1개


In [ ]:
# ===== 1. 라이브러리 임포트 =====
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.llms import Ollama
from langchain_core.prompts import PromptTemplate
from pprint import pprint
from langchain.schema import Document
import torch
# 모든 경고 무시
import warnings
warnings.filterwarnings("ignore")  # 모든 경고 무시


# ✅ 디바이스 설정: GPU 사용 가능하면 GPU, 아니면 CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


# ===== 2. PDF 문서 로드 =====
## PyPDFLoader가 PDF를 페이지 단위 Document 리스트로 로딩
## 각 Document에는 page_content(텍스트)와 metadata(페이지 번호 등)가 포함
## page_label이 있으면 그걸, 없으면 page를 써서 나중에 페이지 추적 
loader = PyPDFLoader(r"C:\Users\Admin\Desktop\RAG\RAG\도로교통법(법률)(제20677호).pdf")
pages = loader.load()


# ===== 3. 문서 분할 (Chunk 최적화) =====
## chunk 크기를 800자로 늘리고, 100자 겹치게 설정하여 문맥 손실을 최소화 => 조항 중 글자 수 가장 긴 거 테스트!!!!!!!!!!!!!!
## separators=["제", "조", "\n\n", "\n"] : 한국어 법령 형식을 고려해 “제…조…” 경계와 줄바꿈 기준으로 최대한 의미 단위로 쪼개려는 의도
## chunk_size=800, chunk_overlap=100 : 800자 청크에 100자 겹침으로 문맥 손실을 줄입니다.
## 결과: 검색 단위가 될 splits 리스트 생성.

text_splitter = RecursiveCharacterTextSplitter(
    separators=["제", "조", "\n\n", "\n"], 
    chunk_size=800, 
    chunk_overlap=100
)

splits = text_splitter.split_documents(pages)




# ===== 4. 임베딩 (다국어 E5) ===== ==> 임베딩 한 결과물 저장해놓고 로드해서 사용하는 방향으로 (시간 단축)!!!!!!!!!!!!!!!!!!!!!
## 모델: intfloat/multilingual-e5-base (E5)
## 프리픽스 컨벤션을 쓰는 게 포인트:
## 문서/패시지 쪽엔 "passage: ...", 쿼리엔 "query: ...".

# pdf를 vetor화 해서 그 값을 로컬에 저장해놔라.
# pdf를 청킹단위로 veterdb에 저장. metadata를 잘 쓰는 게
# 같은 임베딩 모델로 vetordb에서 찾아야 잘 찾아준다. vetor화 한 방식이 다 다르기 때문

from langchain_huggingface import HuggingFaceEmbeddings
from langchain.schema import Document

## encode_kwargs={"normalize_embeddings": True}
## 임베딩을 정규화(단위 벡터)하여 코사인 유사도와 동일하게 취급
## FAISS의 기본 L2 거리도, 단위 벡터라면 사실상 코사인과 등가
## 왜 E5 + “query:/passage:”가 중요한가?
### E5는 “dual-encoder(질의/문서) 용도”로 학습되어 프리픽스로 역할을 명확히 해주면 검색 품질이 좋아집니다.
### normalize_embeddings=True로 코사인 기반 랭킹이 안정화 → FAISS L2와 정합.

embedding = HuggingFaceEmbeddings(
    model_name="intfloat/multilingual-e5-base",
    model_kwargs={"device": "cuda"},
    encode_kwargs={"normalize_embeddings": True},  # 코사인 유사도 안정화 
    # 유사도 종류도 여러개..!!!!!!!!!!!!
)



# ===== 5. 인덱스 (문서에 passage) =====
## splits_e5 = [Document(page_content="passage: " + d.page_content, ...)] : E5 권장 프리픽스 적용 후 인덱싱.
## FAISS.from_documents(splits_e5, embedding) -> 각 청크를 임베딩 → FAISS 인덱스에 저장.
## retriever = vectorstore.as_retriever(search_type="mmr", search_kwargs={"k": 10, "fetch_k": 20})
### MMR(Maximal Marginal Relevance) 리트리버:
### 단순 유사도 Top-K와 달리 다양성을 고려해 중복/유사 청크를 줄이고 대표성을 높입니다.
### fetch_k=20개를 먼저 후보로 뽑고, 그중 상호 중복을 줄이면서 k=10개를 최종 반환.

splits_e5 = [Document(page_content="passage: " + d.page_content, metadata=d.metadata) for d in splits]
vectorstore = FAISS.from_documents(splits_e5, embedding) 
retriever = vectorstore.as_retriever(search_type="mmr", search_kwargs={"k": 10, "fetch_k": 20}) # ranking

# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!1
## 검색 시 유사도로 할지, 키워드로 할지 정해야 => 상의 필요
### 키워드 검색이 필요 없다면 faiss (유사도 기반, 무료)
### 키워드 검색 시 : 파서 중요. 형태소 기반 문서 인덱싱
### 각각의 장점을 가져오기 위해 앙상블 리트리버(가중치 조절로) => 테스트 필요
### 키워드 + 유사도 => 가중치 설정해서 ranking -> retriever 검색
### 결과물 ranking


# ===== 7. 질문(Query) 확장 =====
query = """

The ego vehicle drives in the middle lane of a four-lane road under clear weather. It approaches a bus stop where a pedestrian is walking across the road. The pedestrian suddenly steps onto the road, and the ego vehicle fails to stop in time, causing a collision. The pedestrian is knocked to the ground, and the vehicle continues forward after the impact.

"""

# ===== 8. 관련 문서 검색 (쿼리에 query) =====a
search_result = retriever.get_relevant_documents(f"query: {query.strip()}")



# ===== 9. 검색 결과 미리보기 =====
def pretty_print_documents(doc_list):
    for i, doc in enumerate(doc_list):
        print(f"\n📄 문서 {i+1}")
        print(f"▶ 페이지 번호 : {doc.metadata.get('page_label', doc.metadata.get('page'))}")
        print("▶ 전체 내용:")
        print(doc.page_content)  # 전문 출력
        print("-" * 80)

pretty_print_documents(search_result)



# ===== 10. LLM 모델 (Ollama) =====
llm = Ollama(model="gpt-oss")


# ===== 11. 프롬프트 템플릿 =====
## 마크다운 문법 사용 가능함. xml을 태그로 줄 수 있음

template = """
당신은 도로교통법 전문가입니다.
다음은 도로교통법 문서에서 검색된 일부 조항과 문장입니다:

{context}

아래 도로주행상황설명 텍스트에 대해 도로교통법 조문을 인용해 객관식 문제를 생성하세요.

요구사항:
1. 도로주행상황설명 텍스트는 영상을 시간순으로 보며 운전자 시점에서 벌어지는 상황을 나열한 것입니다. 운전자 시점에서 상황을 이해하세요.
2. 도로 주행 시 운전자가 마주하는 사고 위험 순간이 있습니다. 사고 위험 순간을 리스트로 정리해주세요.
3. 사고 위험 순간 리스트의 케이스를 중 하나를 골라, 만약 문제를 푸는 사람이 운전자라면 사고 위험 순간 직전에 어떻게 행동하는 것이 올바른 행동인지 맞추는 객관식 문제를 만들어주세요.
4. 객관식 문제의 선택지는 정답(운전자의 올바른 행동) 1개, 오답(운전자의 잘못된 행동) 3개로 만들어주세요.
5. 운전자의 올바른 행동은 도로교통법 조문에 근거한 행동이어야 합니다. 운전자의 잘못된 행동은 위법,위협,사고유발 행동이어야 합니다.
6. 문제와 답을 만든 후, 4개의 선택지에 대해 해설을 해주세요. 
7. 정답 선지의 경우, 왜 정답인지 도로교통법 조문을 인용해 "도로교통법 제XX조 제X항 '문장 전체'"를 반드시 그대로 인용 후 정답인 이유를 설명하세요.
8. 오답 선지의 경우, 왜 오답인지 도로교통법 조문을 인용해 "도로교통법 제XX조 제X항 '문장 전체'"를 반드시 그대로 인용 후 오답인 이유를 설명하세요.
9. 위 작업을 사고 위험 순간 리스트에 있는 모든 케이스에 대해 적용해주세요. 케이스 1개 당 객관식 문제 1개를 만들어주세요. 만약 케이스가 8개라면 객관식 문제 8개가 나와야 합니다.
10. 위 요구사항을 전부 지켜주세요.



질문:
{question}
"""


prompt = PromptTemplate.from_template(template)


# ===== 12. 최종 프롬프트 생성 =====
context = "\n\n".join(doc.page_content for doc in search_result)
final_prompt = prompt.format(context=context, question=query)
print("\n\n======= 도로주행설명 텍스트 =======")
print(query.replace(".",".\n"))  # 최종 프롬프트 출력


# ===== 13. Ollama 호출 =====
response = llm.invoke(final_prompt) # temperature 등등 조절하면서 테스트
## 검색된 법령 문구를 그대로 컨텍스트로 넣었기 때문에 인용 정확도가 올라감
## 


# 모델명을 llm 객체의 속성에서 가져오도록 수정
print(f"\n\n======= Ollama 모델 : {llm.model} 응답 =======")
print(response)



# 로직 바꾸기

## 로직 A (위험상황→RAG 일괄 수집→문제+해설)

In [ ]:
# -*- coding: utf-8 -*-
# logic_a.py

import os
import warnings
from pprint import pprint

import torch
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.llms import Ollama
from langchain_core.prompts import PromptTemplate
from langchain.schema import Document

warnings.filterwarnings("ignore")

# ===== 경로/설정 =====
VECTORSTORE_PATH = "./vectorstore_cache"
PDF_PATH = r"도로교통법(법률)(제20677호).pdf"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", DEVICE)

# ===== 임베딩 =====
embedding = HuggingFaceEmbeddings(
    model_name="intfloat/multilingual-e5-base",
    model_kwargs={"device": "cuda"},
    encode_kwargs={"normalize_embeddings": True},
)

# ===== 벡터스토어 =====
if os.path.exists(VECTORSTORE_PATH):
    print("✅ 기존 벡터스토어 로드 중...")
    vectorstore = FAISS.load_local(
        VECTORSTORE_PATH, embedding, allow_dangerous_deserialization=True
    )
else:
    print("⚡ 신규 벡터스토어 생성 중...")
    loader = PyPDFLoader(PDF_PATH)
    pages = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(
        separators=["제", "조", "\n\n", "\n"],
        chunk_size=1200,
        chunk_overlap=200,
    )
    splits = text_splitter.split_documents(pages)

    # E5 권장 프리픽스
    splits_e5 = [
        Document(page_content="passage: " + d.page_content, metadata=d.metadata)
        for d in splits
    ]

    vectorstore = FAISS.from_documents(splits_e5, embedding)
    vectorstore.save_local(VECTORSTORE_PATH)
    print(f"💾 벡터스토어 저장 완료: {VECTORSTORE_PATH}")

retriever = vectorstore.as_retriever(
    search_type="mmr", search_kwargs={"k": 6, "fetch_k": 20}
)

# ===== LLM =====
llm = Ollama(model="gpt-oss")


# =========================
# 헬퍼
# =========================
def extract_dangers_from_query(query: str) -> list[str]:
    """gpt-oss로 사고 위험 상황 리스트(JSON 배열) 추출"""
    template = """
당신은 도로교통법 기반 사고 위험 분석가입니다.
아래 도로주행상황설명 텍스트에서 운전자 시점의 '사고 위험 순간'만 핵심 키워드 중심으로 1~8개 목록으로 뽑아주세요.
- 시간순
- 출력은 문자열 리스트만

텍스트:
{query}
"""
    prompt = PromptTemplate.from_template(template).format(query=query.strip())
    raw = llm.invoke(prompt)

    import json, re
    try:
        match = re.search(r"\[.*\]", str(raw), flags=re.S)
        arr = json.loads(match.group(0)) if match else []
        return [str(x).strip() for x in arr if str(x).strip()]
    except Exception:
        lines = [l.strip("-• \n\r\t") for l in str(raw).splitlines() if l.strip()]
        return [l for l in lines if len(l) > 1][:6]


def retrieve_law_context_for_items(items: list[str], top_k_per_item: int = 3) -> list[Document]:
    """각 항목으로 RAG하고 문서 합침(중복 제거)"""
    seen = set()
    merged_docs: list[Document] = []
    for it in items:
        q = f"query: {it}"
        docs = retriever.get_relevant_documents(q)[:top_k_per_item]
        for d in docs:
            key = (d.metadata.get("page_label", d.metadata.get("page")), d.page_content[:100])
            if key not in seen:
                seen.add(key)
                merged_docs.append(d)
    return merged_docs


def join_docs(docs: list[Document]) -> str:
    parts = []
    for d in docs:
        page = d.metadata.get("page_label", d.metadata.get("page"))
        parts.append(f"[페이지 {page}]\n{d.page_content}")
    return "\n\n".join(parts)


# =========================
# Logic A 본체
# =========================
def run_logic_a(query: str):
    dangers = extract_dangers_from_query(query)
    print("🔎 추출된 사고 위험 상황:")
    pprint(dangers)

    if not dangers:
        print("⚠️ 위험 상황을 추출하지 못했습니다.")
        return

    law_docs = retrieve_law_context_for_items(dangers, top_k_per_item=3)
    context = join_docs(law_docs)

    template = """
당신은 도로교통법 전문가입니다.

# 법령 발췌(context)
{context}

# 사고 위험 순간 목록(dangers)
{dangers}

요구사항:
1) dangers의 각 항목(사고 위험 순간)마다 객관식 문제 1개를 만드세요.
2) 각 문제: 보기 4개(정답 1, 오답 3). 운전자 시점에서 '사고 위험 직전 올바른 행동'을 맞히는 문제.
3) 정답/오답 해설에는 반드시 위 context에 포함된 '도로교통법 제XX조 제X항 "문장 전체"'를 그대로 인용하고, 근거로 왜 정/오답인지 설명.
4) 한국어로 출력. 문제는 1) 2) 3) ... 번호로 구분.
"""
    prompt = PromptTemplate.from_template(template).format(
        context=context,
        dangers="\n".join(f"- {d}" for d in dangers),
    )
    response = llm.invoke(prompt)

    print("\n\n======= Logic A 결과 =======")
    print(response)
    return response



from logic_a_notebook import run_logic_a

query = """

The ego vehicle drive down the middle lane of a three-lane road in clear weather. 
The ego vehicle changes lanes and approaches the bus stop.
A pedestrian suddenly jaywalks into the road, and there is a collision between the ego vehicle and the pedestrian. 
The pedestrian fall to the ground and the ego vehicle stop driving.

"""

result = run_logic_a(query)
print(result)

## 로직 B (위험상황→문제 생성→문제별 RAG→해설)

In [ ]:
# -*- coding: utf-8 -*-
# Jupyter one-cell runner for logic_b
# -------------------------------------------------------------
# 이 노트북 셀은 "도로교통법 PDF"를 벡터화하여 RAG로 참조하고,
# 주어진 도로주행 상황 텍스트에서 "사고 위험 순간"을 추출한 뒤
# 각 위험 상황에 대한 객관식 문제를 생성하고, 법령 문맥을 근거로
# 해설을 출력하는 end-to-end 파이프라인입니다.
# -------------------------------------------------------------


import os
import re
import json
import warnings
from pprint import pprint

import torch
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.llms import Ollama
from langchain_core.prompts import PromptTemplate
from langchain.schema import Document

warnings.filterwarnings("ignore")

# =========================
# 0) 경로/환경 설정
# =========================
VECTORSTORE_PATH = "./vectorstore_cache"  # FAISS 인덱스를 저장/로드할 로컬 폴더
PDF_PATH = r"도로교통법(법률)(제20677호).pdf"  # 참고할 도로교통법 PDF 경로

# CUDA 사용 가능 여부 확인 -> 추론/임베딩 속도에 영향
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", DEVICE)

# PDF 파일이 실제로 존재하는지 사전 점검(없으면 이후 단계에서 실패)
if not os.path.exists(PDF_PATH):
    raise FileNotFoundError(
        f"PDF를 찾을 수 없습니다: {PDF_PATH}\n"
        f"- 노트북과 같은 폴더에 두거나 PDF_PATH를 수정하세요."
    )

# =========================
# 1) 임베딩 모델 준비 (E5)
# =========================
# - multilingual-e5-base: 도메인 범용 다국어 임베딩 모델
# - E5 모델은 입력 텍스트 앞에 'query: ' 또는 'passage: ' 등의 프리픽스를 권장
#   (검색 쿼리/문서의 역할을 명확히 하여 검색 품질 향상)
# - 현재는 device="cuda"로 고정되어 있어 GPU가 없으면 에러가 날 수 있습니다.
#   *CPU만 사용하실 경우* 아래 줄을
#     model_kwargs={"device": "cpu"}
#   로 바꾸시길 권장드립니다.
embedding = HuggingFaceEmbeddings(
    model_name="intfloat/multilingual-e5-base",
    model_kwargs={"device": "cuda"},  # 필요 시 "cpu" 또는 (torch.cuda.is_available() 조건부)로 변경 권장
    encode_kwargs={"normalize_embeddings": True},  # 벡터 정규화(코사인 유사도에 유리)
)

# =========================
# 2) 벡터스토어(FAISS) 로드 또는 신규 생성
# =========================
# - 최초 실행: PDF를 페이지 단위로 로드 → 텍스트 분할 → 임베딩 → FAISS 생성/저장
# - 재실행: 저장된 벡터스토어를 즉시 로드(빠름)
if os.path.exists(VECTORSTORE_PATH):
    print("✅ 기존 벡터스토어 로드 중...")
    # allow_dangerous_deserialization=True:
    #  - 로컬에서 신뢰 가능한 환경이라 가정하고, FAISS 인덱스 역직렬화를 허용
    vectorstore = FAISS.load_local(
        VECTORSTORE_PATH, embedding, allow_dangerous_deserialization=True
    )
else:
    print("⚡ 신규 벡터스토어 생성 중...")
    # 2-1) PDF 로드(페이지 단위로 Document 리스트 생성)
    loader = PyPDFLoader(PDF_PATH)
    pages = loader.load()

    # 2-2) 텍스트 분할
    # - '제', '조' 등의 구분자를 우선 고려하여 조항 단위로 쪼개되
    #   chunk_size/overlap을 조정해 검색/인용 품질을 튜닝
    # - chunk_size가 너무 크면 정확히 필요한 한 문장을 집어오기 어렵고,
    #   너무 작으면 문맥이 끊겨 해설 근거가 빈약해질 수 있습니다.
    text_splitter = RecursiveCharacterTextSplitter(
        separators=["제", "조", "\n\n", "\n"],
        chunk_size=1500,     # 기본 1500자
        chunk_overlap=200,   # 인접 청크 간 200자 겹침(문맥 연속성 보강)
    )
    splits = text_splitter.split_documents(pages)

    # 2-3) E5 모델 권장 프리픽스 적용
    # - 검색 대상 문서엔 'passage: '를 붙여 임베딩
    splits_e5 = [
        Document(page_content="passage: " + d.page_content, metadata=d.metadata)
        for d in splits
    ]

    # 2-4) FAISS 인덱스 생성 및 디스크 저장(다음번부터는 빠르게 로드 가능)
    vectorstore = FAISS.from_documents(splits_e5, embedding)
    vectorstore.save_local(VECTORSTORE_PATH)
    print(f"💾 벡터스토어 저장 완료: {VECTORSTORE_PATH}")

# 2-5) 검색기(retriever) 구성
# - search_type="mmr"(Maximal Marginal Relevance): 유사성과 다양성 균형
# - k: 최종 반환 문서 수, fetch_k: 후보군에서 먼저 많이 뽑아 다양성 최적화
retriever = vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 10, "fetch_k": 20}
)

# =========================
# 3) LLM(Ollama) 준비
# =========================
# - 로컬에서 Ollama가 실행 중이어야 하며 `gpt-oss` 모델이 설치되어 있어야 함
#   (없다면 `ollama pull gpt-oss` 후 `ollama run gpt-oss`로 확인)
# - 기본 포트가 아닐 때는: Ollama(model="gpt-oss", base_url="http://127.0.0.1:11434")
llm = Ollama(model="gpt-oss")

# =========================
# 4) 헬퍼 함수들 (파서/검색/문맥 병합)
# =========================

def _extract_json_array(text: str):
    """
    LLM 출력 본문에서 '최상위 JSON 배열' 형태([ ... ])를 찾아 파싱합니다.
    - 정규식은 대괄호로 감싼 첫 배열을 느슨하게 포착합니다.
    - LLM이 JSON 앞뒤로 설명을 덧붙이는 경우를 대비합니다.
    - 실패 시 None 반환.
    """
    s = str(text)

    # 코드펜스 안쪽 JSON 먼저 시도
    fence_match = re.search(r"```(?:json)?\s*(\[[\s\S]*?\])\s*```", s, flags=re.IGNORECASE)
    if fence_match:
        try:
            return json.loads(fence_match.group(1))
        except Exception:
            pass  # 아래 일반 로직으로 재시도

    # 최상위 '[' 찾기
    start = s.find('[')
    if start == -1:
        return None

    depth = 0
    end = -1
    for i in range(start, len(s)):
        ch = s[i]
        if ch == '[':
            depth += 1
        elif ch == ']':
            depth -= 1
            if depth == 0:
                end = i + 1
                break

    if end == -1:
        return None

    candidate = s[start:end]
    try:
        return json.loads(candidate)
    except Exception:
        return None






def extract_dangers_from_query(query: str) -> list[str]:
    """
    (1단계) 도로주행 상황 설명으로부터 '사고 위험 순간' 리스트를 LLM으로 추출합니다.
    - 출력은 가능한 한 JSON 배열(문자열 리스트) 형태를 기대합니다.
    - JSON 파싱이 실패하면 불릿(•/-) 라인을 폴백으로 일부 수집합니다.
    """
    template = """
당신은 도로교통법 기반 사고 위험 분석가입니다.
아래 도로주행상황설명 텍스트는 영상을 시간순으로 보며 운전자 시점에서 벌어지는 상황을 나열한 것입니다. 운전자 시점에서 상황을 이해하세요.
도로 주행 시 운전자가 마주하는 사고 위험 순간이 있습니다. **사고 위험 순간**을 리스트로 정리해주세요.
'사고 위험 순간'만 핵심 키워드 중심으로 1~8개 목록으로 뽑아주세요.
- 시간순
- 출력은 JSON 배열(문자열 리스트)만

텍스트:
{query}



"""
    prompt = PromptTemplate.from_template(template).format(query=query.strip())
    raw = str(llm.invoke(prompt))  # Ollama LLM 호출 → 문자열 결과

    # 1) JSON 배열 시도
    arr = _extract_json_array(raw)
    if arr and isinstance(arr, list):
        # 각 항목 문자열 정리(양쪽 공백 제거 및 빈 항목 제거)
        return [str(x).strip() for x in arr if str(x).strip()]

    # 2) 실패 시: 줄 단위로 나눠 불릿 라인만 추려서 최대 6개까지 폴백
    lines = [l.strip("-• \n\r\t") for l in str(raw).splitlines() if l.strip()]
    return [l for l in lines if len(l) > 1][:6]

def retrieve_law_context_for_items(items: list[str], top_k_per_item: int = 5) -> list[Document]:
    """
    (2단계-1) 각 항목(질문/선택지 등)을 쿼리로 삼아 RAG 검색을 수행하고,
    중복 문서를 제거한 뒤 합친 리스트를 반환합니다.

    - 'query: ' 프리픽스를 붙여 E5 모델 쿼리 최적화
    - seen 집합으로 (페이지, 문서 앞부분) 키를 사용해 중복 제거
    """
    seen = set()
    merged_docs: list[Document] = []
    for it in items:
        q = f"query: {it}"
        docs = retriever.get_relevant_documents(q)[:top_k_per_item]
        for d in docs:
            key = (
                d.metadata.get("page_label", d.metadata.get("page")),  # 페이지 번호(라벨 없으면 page)
                d.page_content[:100],                                   # 문서 앞 100자(간단 중복 키)
            )
            if key not in seen:
                seen.add(key)
                merged_docs.append(d)
    return merged_docs

def join_docs(docs: list[Document]) -> str:
    """
    (2단계-2) 문서 리스트를 사람이 읽기 쉬운 하나의 문자열로 병합합니다.
    - 각 문서 앞에 [페이지 N] 헤더를 붙여 근거 위치를 추적 가능하게 함
    """
    parts = []
    for d in docs:
        page = d.metadata.get("page_label", d.metadata.get("page"))
        parts.append(f"[페이지 {page}]\n{d.page_content}")
    return "\n\n".join(parts)

# =========================
# 5) Logic B 본체
# =========================
def run_logic_b(query: str):
    """
    전체 파이프라인 실행:
      (1) 위험 상황 추출 → (2) 객관식 문제 생성(JSON) → (3) RAG 문맥 기반 해설 생성
    """

    # (1) LLM으로 '사고 위험 순간' 리스트 추출
    dangers = extract_dangers_from_query(query)
    print("🔎 추출된 사고 위험 상황:")
    pprint(dangers)

    if not dangers:
        print("⚠️ 위험 상황을 추출하지 못했습니다.")
        return

    # (2) 각 위험 상황을 바탕으로 '정답 1, 오답 3'의 객관식 문제를 생성
    #     - 여기서는 해설/법령 인용 없이 문제 JSON만 받습니다.
    make_question_tpl = """
        당신은 도로교통법 기반 객관식 문제 출제자입니다.
        아래 '사고 위험 순간' 각각에 대해, 운전자 시점의 '사고 위험 직전 올바른 행동'을 맞히는 객관식 문제를 1개씩 만드세요.
        - 객관식 문제의 선택지는 정답(운전자의 올바른 행동) 1개, 오답(운전자의 잘못된 행동) 3개로 만들어주세요.
        - 운전자의 올바른 행동은 도로교통법 조문에 근거한 행동이어야 합니다. 운전자의 잘못된 행동은 위법,위협,사고유발 행동이어야 합니다.
        - 아직 해설/법령 인용은 작성하지 마세요
        - 출력은 JSON 배열로, 각 원소는 {{"danger": "...", "question": "...", "choices": ["A. ...","B. ...","C. ...","D. ..."], "answer": "A"}} 형태

        사고 위험 순간 목록:
        {dangers}


        """
    q_prompt = PromptTemplate.from_template(make_question_tpl).format(
        dangers="\n".join(f"- {d}" for d in dangers)
    )
    raw = str(llm.invoke(q_prompt))     # 문제 생성 LLM 호출
    questions = _extract_json_array(raw) or []  # JSON 배열 파싱 실패 시 빈 리스트

    if not questions:
        print("⚠️ 문제 생성을 파싱하지 못했습니다. 원문 출력:")
        print(raw)
        return

    print("\n📝 생성된 문제(요약):")
    for i, q in enumerate(questions, 1):
        print(f"{i}. [{q.get('danger')}] {q.get('question')} / 정답: {q.get('answer')}")

    print("[DEBUG] questions 갯수:", len(questions))
    if not questions:
        print("[DEBUG] 원문:", raw[:1000])


    # (3) 각 문제에 대해: 질문/선택지/정답을 묶어 RAG 검색 → 법령 컨텍스트로 해설 생성
    explain_tpl = """
        당신은 도로교통법 전문가입니다.
        다음은 도로교통법 문서에서 검색된 일부 조항과 문장입니다:

        {context}

        아래 문제에 대해 해설만 작성하세요. 각 선택지에 대해:
        1. 정답 선지의 경우, 왜 정답인지 도로교통법 조문을 인용해 "도로교통법 제XX조 제X항 '문장 전체'"를 반드시 그대로 인용 후 정답인 이유를 설명하세요.
        2. 오답 선지의 경우, 왜 오답인지 도로교통법 조문을 인용해 "도로교통법 제XX조 제X항 '문장 전체'"를 반드시 그대로 인용 후 오답인 이유를 설명하세요.
        3. 답변은 한국어로 해주세요.

        문제:
        {question}
        선택지:
        {choices}
        정답: {answer}
        """

    print("\n\n======= Logic B 결과 =======")
    for idx, q in enumerate(questions, 1):
        # 3-1) 문제/선택지/정답을 하나의 쿼리로 결합 → RAG 검색 강화를 위해 사용
        query_for_rag = f"{q.get('question','')} 선택지: {' '.join(q.get('choices',[]))} 정답: {q.get('answer','')}"
        law_docs = retrieve_law_context_for_items([query_for_rag], top_k_per_item=5)

        print("[DEBUG] law_docs 갯수:", len(law_docs))

        # 3-2) 컨텍스트 생성 (먼저 만들고 나서 미리보기 출력)
        context = join_docs(law_docs)
        preview = (context[:500].replace("\n", " ") + " ...") if context else "(empty)"
        print("[DEBUG] context 미리보기:", preview)

        # 3-3) 해설 생성
        prompt = PromptTemplate.from_template(explain_tpl).format(
            context=context if context else "검색된 법령 문맥이 비어 있습니다.",
            question=q.get("question", ""),
            choices="\n".join(q.get("choices", [])),
            answer=q.get("answer", ""),
        )
        explanation = llm.invoke(prompt)

        # 3-4) 출력 정리
        print(f"\n--- 문제 {idx} ---")
        print(f"[위험상황] {q.get('danger')}")
        print(f"[문제] {q.get('question')}")
        print("[선택지]")
        for c in q.get("choices", []):
            print(c)
        print(f"[정답] {q.get('answer')}")
        print("[해설]")
        print(explanation)


# =========================
# 6) 셀 실행 시 바로 테스트 실행
#    ※ 실제 사용 시, 아래 query 텍스트만 교체하시면 됩니다.
# =========================
query = """

    에고 차량은 3차선 도로의 중간 차선을 주행합니다. 
    에고 차량이 차선을 바꿔 버스 전용차로로 주행합니다.
    보행자가 갑자기 무단횡단하여 버스 전용차로 도로로 들어서고, 자아 차량과 보행자 간의 충돌이 발생합니다. 
    보행자가 바닥에 쓰러지고 에고 차량은 주행을 멈춥니다.

    """
run_logic_b(query)


Using device: cuda
✅ 기존 벡터스토어 로드 중...
🔎 추출된 사고 위험 상황:
['버스 전용 차로로 차선 변경', '보행자 무단횡단', '충돌 발생']

📝 생성된 문제(요약):
1. [버스 전용 차로로 차선 변경] 버스 전용 차로로 차선 변경하려는 상황에서 운전자는 어떻게 행동해야 합니까? / 정답: A
2. [보행자 무단횡단] 보행자가 무단횡단을 하고 있을 때 운전자는 무엇을 해야 합니까? / 정답: A
3. [충돌 발생] 충돌이 발생한 직후 운전자는 무엇을 해야 합니까? / 정답: A
[DEBUG] questions 갯수: 3


======= Logic B 결과 =======
[DEBUG] law_docs 갯수: 5
[DEBUG] context 미리보기: [페이지 19] passage: 제2항에도 불구하고 자전거등의 운전자는 교차로에서 좌회전하려는 경우에는 미리 도로의 우측 가장자리로 붙 어 서행하면서 교차로의 가장자리 부분을 이용하여 좌회전하여야 한다.<개정 2020. 6. 9.> ④ 제1항부터 제3항까지의 규정에 따라 우회전이나 좌회전을 하기 위하여 손이나 방향지시기 또는 등화로써 신호 를 하는 차가 있는 경우에 그 뒤차의 운전자는 신호를 한 앞차의 진행을 방해하여서는 아니 된다. ⑤ 모든 차 또는 노면전차의 운전자는 신호기로 교통정리를 하고 있는 교차로에 들어가려는 경우에는 진행하려는 진로의 앞쪽에 있는 차 또는 노면전차의 상황에 따라 교차로(정지선이 설치되어 있는 경우에는 그 정지선을 넘은 부분을 말한다)에 정지하게 되어 다른 차 또는 노면전차의 통행에 방해가 될 우려가 있는 경우에는 그 교차로에 들 어가서는 아니 된다.<개정 2018. 3. 27.> ⑥ 모든 차의 운전자는 교통정리를 하고 있지 아니하고 일시정지나 양보를 표시 ...

--- 문제 1 ---
[위험상황] 버스 전용 차로로 차선 변경
[문제] 버스 전용 차로로 차선 변경하려는 상황에서 운전자는 어떻게 행동해야 합니까?
[선택지]
A. 현재 차선을 유지하고 버스 전

## 로직 C (도로주행 텍스트 -> RAG 기반 객관식 문제 생성 -> 각각 문제에 대해 RAG 기반 해설 찾기)

In [5]:
# -*- coding: utf-8 -*-
# Jupyter one-cell runner for NEW Logic B (No-JSON version, robust regex parsing)
# ------------------------------------------------------------------------------------
# 1) query로 RAG → 법령 컨텍스트 확보
# 2) LLM이 "플레인 텍스트 스키마"로 위험순간 + 객관식 문항들 생성 (JSON 금지)
# 3) 각 문항 텍스트로 재-RAG → 문항별 해설 생성
# - 출력 파싱은 간단한 정규식 기반(번호 목록 + Q/A 블록)
# - 실패 시 폴백: 최소 1문항이라도 생성하도록 안전장치
# ------------------------------------------------------------------------------------

import os, re, json, warnings
from typing import List, Dict, Tuple
from pprint import pprint

import torch
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.llms import Ollama
from langchain_core.prompts import PromptTemplate
from langchain.schema import Document

warnings.filterwarnings("ignore")

# =========================
# 0) 경로/디바이스 설정
# =========================
VECTORSTORE_PATH = "./vectorstore_cache"
PDF_PATH = r"도로교통법(법률)(제20677호).pdf"
QUIZ_PATH = r"운전면허_문제은행.csv"

DEVICE = "cuda" 
print(f"Using device: {DEVICE}")

if not os.path.exists(PDF_PATH):
    raise FileNotFoundError(f"PDF를 찾을 수 없습니다: {PDF_PATH}")

# =========================
# 1) 임베딩 모델 (E5)
# =========================
embedding = HuggingFaceEmbeddings(
    model_name="intfloat/multilingual-e5-base",
    model_kwargs={"device": DEVICE},
    encode_kwargs={"normalize_embeddings": True},
)

# =========================
# 2) 벡터스토어 로드/생성 (FAISS)
# =========================
if os.path.exists(VECTORSTORE_PATH):
    print("✅ 기존 벡터스토어 로드 중...")
    vectorstore = FAISS.load_local(
        VECTORSTORE_PATH,
        embedding,
        allow_dangerous_deserialization=True
    )
else:
    print("⚡ 신규 벡터스토어 생성 중...")
    loader = PyPDFLoader(PDF_PATH)
    pages = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(
        separators=["제", "조", "\n\n", "\n"],
        chunk_size=1200,
        chunk_overlap=200,
    )
    splits = text_splitter.split_documents(pages)

    splits_e5 = [
        Document(page_content="passage: " + d.page_content, metadata=d.metadata)
        for d in splits
    ]

    vectorstore = FAISS.from_documents(splits_e5, embedding)
    vectorstore.save_local(VECTORSTORE_PATH)
    print(f"💾 벡터스토어 저장 완료: {VECTORSTORE_PATH}")

retriever = vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 10, "fetch_k": 20}
)

# =========================
# 3) LLM 구성 (Ollama)
# =========================
# 이제 JSON 금지 → 두 모델 모두 일반 텍스트 모드
llm_gen = Ollama(model="gpt-oss", temperature=0)  # 문항 생성
llm_exp = Ollama(model="gpt-oss", temperature=0)  # 해설 생성

# =========================
# 4) 유틸/파서
# =========================
def join_docs(docs: List[Document]) -> str:
    parts = []
    for d in docs:
        page = d.metadata.get("page_label", d.metadata.get("page"))
        parts.append(f"[페이지 {page}]\n{d.page_content}")
    return "\n\n".join(parts)

def build_context_from_query(query_text: str) -> str:
    docs = retriever.get_relevant_documents("query: " + query_text.strip())
    if not docs: return ""
    return join_docs(docs)

def retrieve_law_context_for_items(items: List[str], top_k_per_item: int = 5) -> str:
    seen = set()
    merged: List[Document] = []
    for it in items:
        docs = retriever.get_relevant_documents("query: " + it)[:top_k_per_item]
        for d in docs:
            key = (d.metadata.get("page_label", d.metadata.get("page")), d.page_content[:120])
            if key not in seen:
                seen.add(key)
                merged.append(d)
    return join_docs(merged)

# --- 파서: 위험순간/문항(플레인 텍스트) ---
QA_BLOCK_RE = re.compile(
    r"""^Q\s*(\d+)\s*[:：]\s*(?P<q>.+?)\s*
A[)\.]?\s*(?P<A>.+?)\s*
B[)\.]?\s*(?P<B>.+?)\s*
C[)\.]?\s*(?P<C>.+?)\s*
D[)\.]?\s*(?P<D>.+?)\s*
(?:ANS|정답)\s*[:：]\s*(?P<ans>[ABCD])\s*
(?:-{3,}\s*)?""",
    re.IGNORECASE | re.MULTILINE | re.DOTALL
)

def parse_dangers(text: str) -> List[str]:
    """
    [위험순간] 섹션에서 '1. ...' 형태의 라인들을 추출
    """
    # 섹션 경계 찾기
    m = re.search(r"\[위험\s*순간\]|\[위험\s*상황\]|\[DANGERS?\]", text, re.I)
    if not m:
        return []
    start = m.end()
    # 다음 섹션 시작까지(문항/QUESTIONS) 잘라내기
    next_m = re.search(r"\[문항\]|\[문제\]|\[QUESTIONS?\]", text[start:], re.I)
    dangers_block = text[start:] if not next_m else text[start:start+next_m.start()]
    # 번호 라인 추출
    items = []
    for line in dangers_block.splitlines():
        line = line.strip()
        mline = re.match(r"^\d+\.\s*(.+)$", line)
        if mline:
            items.append(mline.group(1).strip())
    return items

def parse_questions(text: str) -> List[Dict]:
    """
    Qn/A/B/C/D/정답 블록을 모두 추출해 리스트로 반환
    """
    results = []
    for m in QA_BLOCK_RE.finditer(text):
        q = {
            "idx": int(m.group(1)),
            "question": re.sub(r"\s+", " ", m.group("q")).strip(),
            "choices": {
                "A": re.sub(r"\s+", " ", m.group("A")).strip(),
                "B": re.sub(r"\s+", " ", m.group("B")).strip(),
                "C": re.sub(r"\s+", " ", m.group("C")).strip(),
                "D": re.sub(r"\s+", " ", m.group("D")).strip(),
            },
            "answer": m.group("ans").upper()
        }
        results.append(q)
    # idx 순 정렬
    results.sort(key=lambda x: x["idx"])
    return results

# =========================
# 5) 프롬프트 (플레인 텍스트 스키마)
# =========================
MAKE_TXT_TPL = """
당신은 도로교통법 전문가 겸 문제 출제자입니다.
아래 '법령 발췌(context)'와 '도로주행상황설명(query)'를 근거로,
JSON, 표, 코드펜스 없이 **아래 형식** 그대로만 출력하세요.

[형식 가이드 - 반드시 그대로]
[위험순간]
1. (운전자 시점 사고 위험 순간) 
2. ...
(최대 8개)

[문항]
Q1: (문제 문장; 한 문장)
A) (선지 A)
B) (선지 B)
C) (선지 C)
D) (선지 D)
정답: (A/B/C/D)
---
Q2: (문제 문장)
A) ...
B) ...
C) ...
D) ...
정답: (A/B/C/D)
(필요 개수만큼 반복; 최소 1문항)

[주의]
- JSON/마크다운/코드블록/표/불릿 금지, 위 형식 외 장식 금지
- 선택지는 정확히 4개(A~D), 정답 1개
- 올바른 행동(정답 선지)은 도로교통법에 근거, 잘못된 행동(오답 선지)은 불법, 위협 운전에 해당하도록
-

[법령 발췌(context)]
{{context}}

[도로주행상황설명(query)]
{{query}}
"""

MAKE_TXT_PROMPT = PromptTemplate(
    template=MAKE_TXT_TPL,
    input_variables=["context", "query"],
    template_format="jinja2"
)

EXPLAIN_TPL = """
당신은 도로교통법 전문가입니다.
아래 문항의 각 선택지에 대해 순수 텍스트로만 해설하세요. (마크다운/표/코드펜스 금지)

요구:
1) 정답: 왜 정답인지, 관련 조문을 "도로교통법 제XX조 제X항 '문장 전체'" 형식으로 1회 이상 정확히 인용 후 설명.
2) 오답: 왜 오답인지, 동일 형식으로 관련 조문 인용 후 설명.
3) 한국어

# 법령 발췌(context)
{context}

문제: {question}
선택지:
A) {A}
B) {B}
C) {C}
D) {D}
정답: {answer}
"""

# =========================
# 6) 실행 함수
# =========================
def run_new_logic_b(query: str):
    # [1] query → 광의 컨텍스트
    base_context = build_context_from_query(query)
    if not base_context:
        print("⚠️ RAG에서 컨텍스트를 찾지 못했습니다. (PDF/인덱스/VECTORSTORE_PATH 확인)")
        return

    # [1-2] 문항(플레인 텍스트) 생성
    gen_prompt = MAKE_TXT_PROMPT.format(context=base_context, query=query.strip())
    gen_text = str(llm_gen.invoke(gen_prompt)).strip()

    # 디버그: 원문 출력(원하면 주석 해제)
    # print("===== 모델 원문 출력 =====")
    # print(gen_text)

    dangers = parse_dangers(gen_text)
    questions = parse_questions(gen_text)

    # 폴백: 문항이 0개면 1문항을 강제로 생성하도록 간단 재요청
    if not questions:
        fallback_tpl = """
다음 상황에 대해 객관식 1문항만 위 형식으로 출력하세요.

[문항]
Q1: (문제 문장)
A) ...
B) ...
C) ...
D) ...
정답: (A/B/C/D)

[법령 발췌(context)]
{context}

[상황]
{query}
"""
        fb_text = str(llm_gen.invoke(fallback_tpl.format(context=base_context, query=query.strip()))).strip()
        questions = parse_questions(fb_text)

    if not questions:
        print("⚠️ 문항을 생성/파싱하지 못했습니다. 모델 출력 확인 필요.")
        print("=== 생성 텍스트 ===")
        print(gen_text[:1500] + ("... [truncated]" if len(gen_text) > 1500 else ""))
        return

    # 요약 출력
    print("🧭 위험순간(파싱 결과):")
    if dangers:
        for i, d in enumerate(dangers, 1):
            print(f"{i}. {d}")
    else:
        print("(위험순간 섹션을 찾지 못했거나 비어 있음)")

    print("\n📝 생성 문항(요약):")
    for q in questions:
        print(f"Q{q['idx']} | 정답={q['answer']} | {q['question'][:60]}...")

    # [2] 각 문항을 다시 질의로 재-RAG → 해설 생성
    print("\n\n======= 최종 결과 =======")
    for q in questions:
        question_text = q["question"]
        choices_dict = q["choices"]
        answer_key = q["answer"]

        # 재-RAG 질의
        query_for_rag = f"{question_text} 선택지: A){choices_dict['A']} B){choices_dict['B']} C){choices_dict['C']} D){choices_dict['D']} 정답:{answer_key}"
        per_question_context = retrieve_law_context_for_items([query_for_rag], top_k_per_item=5)

        # 해설 프롬프트
        exp_prompt = EXPLAIN_TPL.format(
            context=per_question_context,
            question=question_text,
            A=choices_dict["A"],
            B=choices_dict["B"],
            C=choices_dict["C"],
            D=choices_dict["D"],
            answer=answer_key
        )
        explanation = llm_exp.invoke(exp_prompt)

        # 출력
        print(f"\n--- 문제 {q['idx']} ---")
        print(f"[문제] {question_text}")
        print("A)", choices_dict["A"])
        print("B)", choices_dict["B"])
        print("C)", choices_dict["C"])
        print("D)", choices_dict["D"])
        print(f"[정답] {answer_key}")
        print("[해설]")
        print(explanation)

# =========================
# 7) 실행 예시 (셀 하단에서 바로 실행)
# =========================
query = """
에고 차량은 3차선 도로의 중간 차선을 주행합니다.
에고 차량이 차선을 바꿔 버스 전용차로로 주행합니다.
보행자가 갑자기 무단횡단하여 버스 전용차로 도로로 들어서고, 자아 차량과 보행자 간의 충돌이 발생합니다.
보행자가 바닥에 쓰러지고 에고 차량은 주행을 멈춥니다.
"""
run_new_logic_b(query)


Using device: cuda
✅ 기존 벡터스토어 로드 중...


KeyboardInterrupt: 

# 꼬리 문제 형태로
- 상황 설명 텍스트 -> 문제 1개 만들기
- 상황 설명 텍스트에 나온 키워드 기반으로 키워드와 관련된 퀴즈 만들기

In [ ]:
# 코드 작성

# 조 단위로 벡터 저장

- 조(條)” 단위로 부모 문서를 만들고, 그 안을 작은 청크(자식)로 쪼개서 검색은 자식으로 하되 **반환은 항상 ‘해당 조 전체(부모)’**가 오도록

In [ ]:
# -*- coding: utf-8 -*-
# Jupyter one-cell runner for logic_b (조(條) 단위 Parent Retrieval 통합)
# -------------------------------------------------------------
# - PDF를 '제N조' 단위(부모)로 파싱하여 원문을 보존
# - 각 조 내부를 청킹(자식) → FAISS 색인(자식에는 E5 'passage:' 프리픽스)
# - 검색 시: MMR로 '자식'을 찾고 → 해당 '부모(조 전체)'를 반환
# - 나머지 파이프라인(위험 순간 추출, 문제 생성, RAG 해설)은 동일
# -------------------------------------------------------------

import os
import re
import json
import warnings
from pprint import pprint
from typing import List, Dict, Tuple

import torch
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter  # langchain>=0.2
from langchain_community.vectorstores import FAISS
from langchain_community.llms import Ollama
from langchain_core.prompts import PromptTemplate
from langchain.schema import Document

warnings.filterwarnings("ignore")

# =========================
# 0) 경로/환경 설정
# =========================
VECTORSTORE_PATH = "./vectorstore_cache"           # FAISS 인덱스 저장 폴더
PARENT_STORE_PATH = "./parent_articles_store.json" # ★ 부모(조) 원문 저장 파일
PDF_PATH = r"도로교통법(법률)(제20677호).pdf"

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", DEVICE)

if not os.path.exists(PDF_PATH):
    raise FileNotFoundError(
        f"PDF를 찾을 수 없습니다: {PDF_PATH}\n"
        f"- 노트북과 같은 폴더에 두거나 PDF_PATH를 수정하세요."
    )

# =========================
# 1) 임베딩 모델 (E5)
# =========================
embedding = HuggingFaceEmbeddings(
    model_name="intfloat/multilingual-e5-base",
    model_kwargs={"device": "cuda" if torch.cuda.is_available() else "cpu"},
    encode_kwargs={"normalize_embeddings": True},
)

# =========================
# 2) '제N조' 단위 파서 (부모 생성) ★
# =========================
ARTICLE_PATTERN = re.compile(
    r"(제\s*\d+\s*조(?:\s*\[[^\]]+\])?(?:\s*\([^)]+\))?)\s*(.*?)(?=제\s*\d+\s*조|\Z)",
    re.DOTALL
)

def load_articles_as_parents(pdf_path: str) -> List[Document]:
    """PDF 전체를 이어 붙여 '제N조' 단위로 부모 문서를 만든다."""
    loader = PyPDFLoader(pdf_path)
    pages = loader.load()
    full_text = "".join(p.page_content for p in pages)

    parents: List[Document] = []
    for m in ARTICLE_PATTERN.finditer(full_text):
        header = m.group(1).strip()   # "제1조(목적)" 등
        body   = m.group(2).strip()

        num_m = re.search(r"제\s*(\d+)\s*조", header)
        title_m = re.search(r"제\s*\d+\s*조\s*\(([^)]+)\)", header)
        article_no = int(num_m.group(1)) if num_m else None
        article_title = title_m.group(1) if title_m else None

        parent_id = f"article_{article_no if article_no is not None else header}"

        parents.append(
            Document(
                page_content=f"{header}\n{body}",
                metadata={
                    "unit": "article",
                    "article_no": article_no,
                    "article_header": header,
                    "article_title": article_title,
                    "parent_id": parent_id,
                }
            )
        )
    return parents

# =========================
# 3) 부모 저장/로드 유틸 ★
# =========================
def save_parents(parents: List[Document], path: str):
    data = []
    for d in parents:
        data.append({
            "page_content": d.page_content,
            "metadata": d.metadata
        })
    with open(path, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

def load_parents(path: str) -> Dict[str, Document]:
    with open(path, "r", encoding="utf-8") as f:
        data = json.load(f)
    parents_dict = {}
    for item in data:
        meta = item.get("metadata", {})
        pid = meta.get("parent_id")
        if pid:
            parents_dict[pid] = Document(
                page_content=item.get("page_content", ""),
                metadata=meta
            )
    return parents_dict

# =========================
# 4) 인덱스 준비 (자식 청크 생성 + FAISS) ★
# =========================
def build_child_chunks_from_parents(parents: List[Document]) -> List[Document]:
    """
    각 부모(조)를 청킹하여 '자식' Document 리스트를 만든다.
    - 자식 텍스트에는 E5 'passage:' 프리픽스 적용
    - 메타데이터에 parent_id/조번호/헤더 등을 계승
    """
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=800,
        chunk_overlap=120,
        separators=["\n\n", "\n", " "]
    )
    childs: List[Document] = []
    for p in parents:
        sub_docs = splitter.split_text(p.page_content)
        for idx, chunk in enumerate(sub_docs):
            childs.append(
                Document(
                    page_content="passage: " + chunk,  # ★ E5 권장 프리픽스
                    metadata={
                        **p.metadata,
                        "unit": "article_child",
                        "child_idx": idx
                    }
                )
            )
    return childs

# 벡터스토어/부모 스토어 준비
if os.path.exists(VECTORSTORE_PATH) and os.path.exists(PARENT_STORE_PATH):
    print("✅ 기존 벡터스토어 + 부모조 저장소 로드 중...")
    vectorstore = FAISS.load_local(
        VECTORSTORE_PATH, embedding, allow_dangerous_deserialization=True
    )
    parents_dict = load_parents(PARENT_STORE_PATH)  # parent_id -> Document
else:
    print("⚡ 신규 인덱스 생성 중...(조 단위 파싱 → 자식 청킹 → FAISS 색인)")
    parents = load_articles_as_parents(PDF_PATH)         # 부모(조)
    child_chunks = build_child_chunks_from_parents(parents)  # 자식(청크)

    # FAISS 색인 생성 (자식)
    vectorstore = FAISS.from_documents(child_chunks, embedding)
    vectorstore.save_local(VECTORSTORE_PATH)
    print(f"💾 벡터스토어 저장 완료: {VECTORSTORE_PATH}")

    # 부모 저장(JSON)
    save_parents(parents, PARENT_STORE_PATH)
    parents_dict = {d.metadata["parent_id"]: d for d in parents}
    print(f"💾 부모(조) 저장 완료: {PARENT_STORE_PATH}")

# =========================
# 5) '부모 반환' 래퍼 리트리버 ★
# =========================
class ParentByArticleRetriever:
    """
    - 내부적으로 FAISS에서 '자식'을 MMR로 검색
    - 결과의 parent_id를 모아 '부모(조 전체)' Document를 반환
    """
    def __init__(self, vectorstore: FAISS, parents: Dict[str, Document],
                 k: int = 10, fetch_k: int = 20):
        self.vectorstore = vectorstore
        self.parents = parents
        self.k = k
        self.fetch_k = fetch_k

    def get_relevant_documents(self, query: str) -> List[Document]:
        q = query if query.strip().startswith("query:") else ("query: " + query.strip())
        # MMR 검색 (다양성 확보)
        try:
            child_docs = self.vectorstore.max_marginal_relevance_search(
                q, k=self.k, fetch_k=self.fetch_k
            )
        except Exception:
            # 호환 안 되면 일반 유사도 검색으로 폴백
            child_docs = self.vectorstore.similarity_search(q, k=self.k)

        parent_ids = []
        seen = set()
        for d in child_docs:
            pid = d.metadata.get("parent_id")
            if pid and pid not in seen and pid in self.parents:
                seen.add(pid)
                parent_ids.append(pid)

        # 부모(조 전체) 반환
        return [self.parents[pid] for pid in parent_ids]

# 기존 retriever 대체
retriever = ParentByArticleRetriever(vectorstore, parents_dict, k=10, fetch_k=20)

# =========================
# 6) LLM(Ollama)
# =========================
llm = Ollama(model="gpt-oss")

# =========================
# 7) 헬퍼(기존 코드 유지)
# =========================
def _extract_json_array(text: str):
    s = str(text)
    fence_match = re.search(r"```(?:json)?\s*(\[[\s\S]*?\])\s*```", s, flags=re.IGNORECASE)
    if fence_match:
        try:
            return json.loads(fence_match.group(1))
        except Exception:
            pass
    start = s.find('[')
    if start == -1:
        return None
    depth = 0
    end = -1
    for i in range(start, len(s)):
        ch = s[i]
        if ch == '[':
            depth += 1
        elif ch == ']':
            depth -= 1
            if depth == 0:
                end = i + 1
                break
    if end == -1:
        return None
    candidate = s[start:end]
    try:
        return json.loads(candidate)
    except Exception:
        return None

def extract_dangers_from_query(query: str) -> List[str]:
    template = """
당신은 도로교통법 기반 사고 위험 분석가입니다.
아래 도로주행상황설명 텍스트는 영상을 시간순으로 보며 운전자 시점에서 벌어지는 상황을 나열한 것입니다. 운전자 시점에서 상황을 이해하세요.
도로 주행 시 운전자가 마주하는 사고 위험 순간이 있습니다. **사고 위험 순간**을 리스트로 정리해주세요.
'사고 위험 순간'만 핵심 키워드 중심으로 1~8개 목록으로 뽑아주세요.
- 시간순
- 출력은 JSON 배열(문자열 리스트)만

텍스트:
{query}
"""
    prompt = PromptTemplate.from_template(template).format(query=query.strip())
    raw = str(llm.invoke(prompt))
    arr = _extract_json_array(raw)
    if arr and isinstance(arr, list):
        return [str(x).strip() for x in arr if str(x).strip()]
    lines = [l.strip("-• \n\r\t") for l in str(raw).splitlines() if l.strip()]
    return [l for l in lines if len(l) > 1][:6]

def retrieve_law_context_for_items(items: List[str], top_k_per_item: int = 5) -> List[Document]:
    """
    (부모 반환 리트리버 사용) 각 항목을 쿼리로 삼아 RAG 검색.
    - 중복 부모(같은 조)는 제거
    """
    seen = set()
    merged_docs: List[Document] = []
    for it in items:
        q = f"query: {it}"
        docs = retriever.get_relevant_documents(q)[:top_k_per_item]
        for d in docs:
            key = d.metadata.get("parent_id") or d.metadata.get("article_no")
            if key not in seen:
                seen.add(key)
                merged_docs.append(d)
    return merged_docs

def join_docs(docs: List[Document]) -> str:
    """부모(조) 문서를 사람이 읽기 좋게 병합."""
    parts = []
    for d in docs:
        header = d.metadata.get("article_header", "")
        no = d.metadata.get("article_no", "")
        parts.append(f"[{header or f'제{no}조'}]\n{d.page_content}")
    return "\n\n".join(parts)

# =========================
# 8) Logic B 본체 (기존 유지)
# =========================
def run_logic_b(query: str):
    dangers = extract_dangers_from_query(query)
    print("🔎 추출된 사고 위험 상황:")
    pprint(dangers)

    if not dangers:
        print("⚠️ 위험 상황을 추출하지 못했습니다.")
        return

    make_question_tpl = """
당신은 도로교통법 기반 객관식 문제 출제자입니다.
아래 '사고 위험 순간' 각각에 대해, 운전자 시점의 '사고 위험 직전 올바른 행동'을 맞히는 객관식 문제를 1개씩 만드세요.
- 객관식 문제의 선택지는 정답(운전자의 올바른 행동) 1개, 오답(운전자의 잘못된 행동) 3개로 만들어주세요.
- 운전자의 올바른 행동은 도로교통법 조문에 근거한 행동이어야 합니다. 운전자의 잘못된 행동은 위법,위협,사고유발 행동이어야 합니다.
- 아직 해설/법령 인용은 작성하지 마세요
- 출력은 JSON 배열로, 각 원소는 {"danger": "...", "question": "...", "choices": ["A. ...","B. ...","C. ...","D. ..."], "answer": "A"} 형태

사고 위험 순간 목록:
{dangers}
"""
    q_prompt = PromptTemplate.from_template(make_question_tpl).format(
        dangers="\n".join(f"- {d}" for d in dangers)
    )
    raw = str(llm.invoke(q_prompt))
    questions = _extract_json_array(raw) or []
    if not questions:
        print("⚠️ 문제 생성을 파싱하지 못했습니다. 원문 출력:")
        print(raw)
        return

    print("\n📝 생성된 문제(요약):")
    for i, q in enumerate(questions, 1):
        print(f"{i}. [{q.get('danger')}] {q.get('question')} / 정답: {q.get('answer')}")

    explain_tpl = """
당신은 도로교통법 전문가입니다.
다음은 도로교통법 문서에서 검색된 일부 조항(조 단위 원문)입니다:

{context}

아래 문제에 대해 해설만 작성하세요. 각 선택지에 대해:
1. 정답 선지의 경우, 왜 정답인지 도로교통법 조문을 인용해 "도로교통법 제XX조 제X항 '문장 전체'"를 반드시 그대로 인용 후 정답인 이유를 설명하세요.
2. 오답 선지의 경우, 왜 오답인지 도로교통법 조문을 인용해 "도로교통법 제XX조 제X항 '문장 전체'"를 반드시 그대로 인용 후 오답인 이유를 설명하세요.
3. 답변은 한국어로 해주세요.

문제:
{question}
선택지:
{choices}
정답: {answer}
"""

    print("\n\n======= Logic B 결과 =======")
    for idx, q in enumerate(questions, 1):
        query_for_rag = f"{q.get('question','')} 선택지: {' '.join(q.get('choices',[]))} 정답: {q.get('answer','')}"
        law_docs = retrieve_law_context_for_items([query_for_rag], top_k_per_item=5)
        print("[DEBUG] law_docs(조 단위) 갯수:", len(law_docs))

        context = join_docs(law_docs)
        preview = (context[:500].replace("\n", " ") + " ...") if context else "(empty)"
        print("[DEBUG] context 미리보기:", preview)

        prompt = PromptTemplate.from_template(explain_tpl).format(
            context=context if context else "검색된 법령 문맥이 비어 있습니다.",
            question=q.get("question", ""),
            choices="\n".join(q.get("choices", [])),
            answer=q.get("answer", ""),
        )
        explanation = llm.invoke(prompt)

        print(f"\n--- 문제 {idx} ---")
        print(f"[위험상황] {q.get('danger')}")
        print(f"[문제] {q.get('question')}")
        print("[선택지]")
        for c in q.get("choices", []):
            print(c)
        print(f"[정답] {q.get('answer')}")
        print("[해설]")
        print(explanation)

# =========================
# 9) 테스트 실행
# =========================
query = """
에고 차량은 3차선 도로의 중간 차선을 주행합니다. 
에고 차량이 차선을 바꿔 버스 전용차로로 주행합니다.
보행자가 갑자기 무단횡단하여 버스 전용차로 도로로 들어서고, 자아 차량과 보행자 간의 충돌이 발생합니다. 
보행자가 바닥에 쓰러지고 에고 차량은 주행을 멈춥니다.
"""
run_logic_b(query)
